In [1]:
import pandas as pd
import time

In [2]:
# read raw data (downloaded from http://web.mta.info/developers/turnstile.html) to dataframe 

df_march_3 = pd.read_csv('3_3_2018.csv')
df_march_10 = pd.read_csv('3_10_2018.csv')
df_march_17 = pd.read_csv('3_17_2018.csv')
df_march_24 = pd.read_csv('3_24_2018.csv')
df_march_31 = pd.read_csv('3_31_2018.csv')
df_april_7 = pd.read_csv('4_7_2018.csv')
df_april_14 = pd.read_csv('4_14_2018.csv')
df_april_21 = pd.read_csv('4_21_2018.csv')
df_april_28 = pd.read_csv('4_28_2018.csv')
df_may_5 = pd.read_csv('5_5_2018.csv')
df_may_12 = pd.read_csv('5_12_2018.csv')
df_may_19 = pd.read_csv('5_19_2018.csv')
df_may_26 = pd.read_csv('5_26_2018.csv')
df_june_2 = pd.read_csv('6_2_2018.csv')
df_june_9 = pd.read_csv('6_9_2018.csv')
df_june_16 = pd.read_csv('6_16_2018.csv')

In [3]:
# combine individual dataframes into a single merged dataframes and save as csv

frames = [df_march_3, df_march_10, df_march_17, df_march_24, df_march_31, df_april_7, df_april_14, df_april_21, df_april_28, df_may_5, df_may_12, df_may_19, df_may_26, df_june_2, df_june_9, df_june_16]
merge = pd.concat(frames)
merge.to_csv('merged_raw_mta_data.csv', index = False)

In [4]:
# remove white space from column names (specifically EXITS) and create new columns in merge dataframe

new_names = [name.strip() for name in merge.columns]
merge.columns = new_names

In [5]:
# Remove "RECOVR AUD" entries (in DESC column), which refers to a missed audit that was recovered

merge = merge[merge['DESC'] != 'RECOVR AUD']

In [6]:
# convert DATE column to datetime
# create 'month' and 'day_of_week' columns

merge['DATE'] = pd.to_datetime(merge['DATE'])
merge['month'] = merge['DATE'].dt.month
merge['day_of_week'] = merge['DATE'].dt.weekday_name

In [7]:
# compute values for number of people entering and exiting a turnstile for a given time stamp

merge['Ent'] = merge['ENTRIES'].diff()
merge['Ext'] = merge['EXITS'].diff()

In [8]:
# remove NaN values in row 0

merge = merge.dropna(axis=0)

In [9]:
# remove negative values for Ent or Ext

merge = merge[merge['Ent'] >= 0]
merge = merge[merge['Ext'] >= 0]

In [10]:
# Use Dan's magic_number function to remove outliers
# **replace number with Dan's function!

merge = merge[merge['Ent'] <= 312943]
merge = merge[merge['Ext'] <= 143941]

In [11]:
# Sum Ent and Ext to compute total number of people moving through a single subway station in a given time frame

merge['sum_ent_ext'] = merge['Ent'] + merge['Ext']

In [12]:
merge.describe()

,ENTRIES,EXITS,month,Ent,Ext,sum_ent_ext
count,3.035467e+06,3.035467e+06,3.035467e+06,3.035467e+06,3.035467e+06,3.035467e+06
mean,2.639855e+07,1.924189e+07,4.181935e+00,2.472471e+02,1.689993e+02,4.162465e+02
std,1.526510e+08,1.315835e+08,1.109653e+00,3.375615e+03,1.621303e+03,4.613309e+03
min,0.000000e+00,0.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.931220e+05,2.401820e+05,3.000000e+00,1.000000e+01,8.000000e+00,3.000000e+01
50%,2.498978e+06,1.405553e+06,4.000000e+00,7.700000e+01,5.200000e+01,1.760000e+02
75%,6.686968e+06,4.664852e+06,5.000000e+00,2.480000e+02,1.670000e+02,4.660000e+02
max,2.115507e+09,2.049871e+09,6.000000e+00,3.126280e+05,1.438600e+05,4.366260e+05


In [13]:
# move cleaned data to a csv file
merge.to_csv('clean_merged_mta_data.csv', index = False)